Importing required Packages

In [4]:
import io
import random
import requests
import string
import warnings
import numpy as np
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('popular', quiet=True)
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Web-Scrapping for the Data to feed into Bot

In [5]:
url = "https://en.wikipedia.org/wiki/Natural_language_processing"
response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    raw = soup.get_text(separator=" ")
else:
    print("Failed to retrieve the webpage")

Chat Bot

In [6]:
sent_tokens = nltk.sent_tokenize(raw)
word_tokens = nltk.word_tokenize(raw)

In [7]:
lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [8]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):

    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [9]:
def response(user_response):
    bot_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        bot_response=bot_response+"I am sorry! I don't understand you"
        return bot_response
    else:
        bot_response = bot_response+sent_tokens[idx]
        return bot_response

In [10]:
flag=True
print("\nBOT: Hello, I will answer your queries about NLP. If you want to exit, type Bye!")
while(flag==True):
    user_response = input("\nYOU: ")
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("\nBOT: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("\nBOT: "+greeting(user_response))
            else:
                print("\nBOT: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("\nBOT: Bye! take care..")
        break


BOT: Hello, I will answer your queries about NLP. If you want to exit, type Bye!

YOU: hi

BOT: hi

YOU: tell about nlp

BOT: Intro to NLP in Machine Learning" .

YOU: tell me about  Georgetown experiment

BOT: 1950s : The  Georgetown experiment  in 1954 involved fully  automatic translation  of more than sixty Russian sentences into English.

YOU: tell me about Lemmatization

BOT: Lemmatization is another technique for reducing words to their normalized form.

YOU: bye

BOT: Bye! take care..
